# Conversion of VSOP2013 data files to C++

In [1]:
def render_cpp(n):
    import fortranformat as ff

    if not isinstance(n, int) or n < 1 or n > 9:
        raise ValueError("n must be an integer in the [1, 9] range")

    # The fortran format strings for the headers
    # and the terms.
    header_rr = ff.FortranRecordReader("(9x,3i3,i7)")
    term_rr = ff.FortranRecordReader("(i5,1x,4i3,1x,5i3,1x,4i4,1x,i6,1x,3i3,2(f20.16,1x,i3))")

    # Load the data file for planet n.
    with open("VSOP2013p{}.dat".format(n), "r") as f:
        txt = f.readlines()
    
    # Init the return value. This will be a multidimensional
    # jagged array in which the dimensions mean, in order:
    #
    # - the variable index,
    # - the value of the time power alpha,
    # - the term's index.
    #
    # Thus, for instance:
    #
    # ret[0][1][42]
    #
    # means to get the term at index 42
    # for the time power alpha = 1 for the
    # first variable (index 0).
    ret = list([] for i in range(6))
    
    cur_idx = 0
    while True:
        # Extract the variable index and the number of terms
        # for the current section from the header.
        _, v_idx, tpow, nterms = header_rr.read(txt[cur_idx])
      
        # Read the terms.
        tmp = []
        for j in range(nterms):
            # Parse the term.
            rec = term_rr.read(txt[cur_idx + j + 1])
            assert(rec[0] == j + 1)
            
            # Transform the term:
            # - drop the first element (it's just
            #   the term index),
            # - compress the last 4 terms into 2
            #   (the last 4 terms contain the mantissa
            #   and exponent separately).
            new_rec = rec[1:18]
            new_rec.append(float("{:.17g}e{}".format(rec[18],rec[19])))
            new_rec.append(float("{:.17g}e{}".format(rec[20],rec[21])))
            assert(len(new_rec) == 19)
            
            tmp.append(new_rec)

        # Add them to ret.
        assert(len(ret[v_idx - 1]) == tpow)
        ret[v_idx - 1].append(tmp)
        
        # Update cur_idx, exit if we are
        # at the end.
        cur_idx = cur_idx + nterms + 1
        if cur_idx == len(txt):
            break
    
    # Create the contents for the header.
    header_ret = "namespace heyoka::detail\n{\n\n"
    for coord_idx in range(6):
        header_ret += "inline constexpr unsigned long vsop2013_{}_{}_sizes[{}] = {{{}}};\n".format(n, coord_idx+1, len(ret[coord_idx]), ", ".join("{}ul".format(len(_)) for _ in ret[coord_idx]))
    header_ret += "\n"
    for coord_idx in range(6):
        header_ret += "extern const double * const vsop2013_{}_{}_data[{}];\n".format(n, coord_idx + 1, len(ret[coord_idx]))
    header_ret +="\n}\n"
    
    # Create the contents for the cpps.
    cpp_ret = list("namespace heyoka::detail\n{\n\nnamespace\n{\n\n" for _ in range(6))
    for coord_idx in range(6):
        for j in range(len(ret[coord_idx])):
            cpp_ret[coord_idx] += "const double vsop2013_{}_{}_{}[{}ul * 19ul] = ".format(n, coord_idx + 1, j, len(ret[coord_idx][j]))
            cpp_ret[coord_idx] += "{"
            
            nrecs = len(ret[coord_idx][j])
            
            for k in range(nrecs):
                rec = ret[coord_idx][j][k]
                cpp_ret[coord_idx] += "{}, {:.17g}, {:.17g}".format(", ".join(str(_) for _ in rec[:17]), rec[17], rec[18])
                if k != nrecs - 1:
                    cpp_ret[coord_idx] += ", "
            
            cpp_ret[coord_idx] += "};\n\n";
        cpp_ret[coord_idx] +="}\n\n"
    
    for coord_idx in range(6):
        cpp_ret[coord_idx] += "const double * const vsop2013_{}_{}_data[{}] = {{{}}};\n".format(n, coord_idx + 1, len(ret[coord_idx]), ", ".join(["vsop2013_{}_{}_{}".format(n, coord_idx + 1, j) for j in range(len(ret[coord_idx]))]))
        cpp_ret[coord_idx] += "\n}\n"
    
    return header_ret, cpp_ret

In [8]:
pl_idx = 9

h_file, cpp_files = render_cpp(pl_idx)

with open("vsop2013_{}.hpp".format(pl_idx), "w") as f:
    f.write(h_file)

for coord_idx in range(6):
    with open("vsop2013_{}_{}.cpp".format(pl_idx, coord_idx + 1), "w") as f:
        f.write(cpp_files[coord_idx])